In [1]:
import simpy
from lib.core import NetworkTap, Switch, PacketSource, PacketSink, visualize_topology
from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

env = simpy.Environment()

# Create a tap to monitor traffic
tap = NetworkTap(env)

switch = Switch(env, num_ports=2, port_capacity=10, port_transmission_rate=1)
# Attach tap to the first port of the switch
switch.ports[0].tap = tap

# The rest remains the same:
source_A = PacketSource(env, switch, source_id='A', destination='B', interval=2, packet_size=1)
source_B = PacketSource(env, switch, source_id='B', destination='A', interval=3, packet_size=1)

env.process(source_A.start())
env.process(source_B.start())

env.run(until=50)

# Display tap statistics at the end
tap.display_statistics()


In [2]:
env = simpy.Environment()

# Create 3 switches
switch_A = Switch(env, "A", num_ports=4, port_capacity=10, port_transmission_rate=1)
switch_B = Switch(env, "B", num_ports=4, port_capacity=10, port_transmission_rate=1)
switch_C = Switch(env, "C", num_ports=4, port_capacity=10, port_transmission_rate=1)

# Connect A to B
switch_A.ports[0].destination = switch_B

# Connect B to C
switch_B.ports[0].destination = switch_C

# Connect C back to A
switch_C.ports[0].destination = switch_A

# Add a network tap to monitor traffic between A and B
tap = NetworkTap(env)
switch_A.ports[0].tap = tap

# Create two packet sources
source1 = PacketSource(env, "Source1", destination=switch_A, packet_interval=2, packet_size=10)
source2 = PacketSource(env, "Source2", destination=switch_B, packet_interval=3, packet_size=15)


packet_sink = PacketSink(env, "Sink1")
switch_C.ports[1].destination = packet_sink

env.process(source1.start())
env.process(source2.start())

# Run the simulation
simulation_time = 20
env.run(until=simulation_time)



In [3]:
tap.display_statistics()

----- Network Tap Statistics -----
Total Packets Observed: 0
Total Size of Packets: 0 units
Average Packet Size: 0 units
Average Inter-Arrival Time: N/A
------------------------------


In [4]:
packet_sink.display_statistics()

In [5]:
list_of_switches = [switch_A, switch_B, switch_C]
visualize_topology(list_of_switches, [source1, source2], [packet_sink])

ValueError: failed to validate StaticLayoutProvider(id='p1022', ...).graph_layout: expected an element of Dict(Int, Seq(Any)), got {'Src: Source1': (0.465755865495281, -0.799590587231214), 'Src: Source2': (0.3706976920991402, 0.823395213781704), 'A': (0.35445241881001494, -0.6578136881054585), 'B': (0.5441146435080929, 0.8272394634781741), 'C': (-0.7350206199125295, -0.8096616957519897), 'Sink: Sink1': (-1.0, 0.6164312938287838)}